<a href="https://colab.research.google.com/github/cimbelli/Surface-Heat-Islands/blob/main/day/temperatura_max_mensile_capoluoghi-regione_1.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Stima delle temperatura massima mensile nei capoluoghi di regione

In [3]:
import os
import ee
import pandas as pd
import json
import geopandas as gpd
import requests
#ee.Authenticate()
ee.Initialize()
#import geemap
#from geemap import geojson_to_ee, zonal_statistics
from rasterstats import zonal_stats

In [5]:
anno = 2023
mesi = [6,7,8,9]

In [6]:
import folium
from folium import plugins

def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
    ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

In [7]:
def scarica(lyr, name, aoi):
    url = lyr.getDownloadUrl({
        'name': name[:-4],
        'bands': ['ST'],
        'region': aoi,
        'scale': 100,
        'format': 'GEO_TIFF'
    })
    response = requests.get(url)
    with open(name, 'wb') as fd:
        fd.write(response.content)
    return None

In [8]:
# Add custom basemaps to folium
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}

In [14]:
# carica lo shape dei capoluoghi di regione

g = open('comregll2.geojson')
cc = json.load(g)

gdf = gpd.read_file('comcapll.geojson')
gdf["lon"] = gdf["geometry"].centroid.x
gdf["lat"] = gdf["geometry"].centroid.y

C:\Users\cimbe\AppData\Local\Temp\ipykernel_19480\2779875742.py:7: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf["lon"] = gdf["geometry"].centroid.x
C:\Users\cimbe\AppData\Local\Temp\ipykernel_19480\2779875742.py:8: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf["lat"] = gdf["geometry"].centroid.y


In [10]:
for i in range(len(cc['features'])):
    #nome1 = cc['features'][i]['properties']['COMUNE']
    reg = cc['features'][i]['properties']['REG']
    reg1 = ('0' if reg < 10 else '') + str(reg)
    #if nome1 == 'Roma':
    id1 = cc['features'][i]['properties']['PRO_COM']

    coords = cc['features'][i]['geometry']['coordinates']
    fc = ee.FeatureCollection(cc['features'])
    comune = cc['features'][i]['properties']['COMUNE']
    
    g1 = open(os.path.join('sez11','R' + reg1 + '_11geo.geojson')) 
    df_sez11 = gpd.read_file(g1)
    df_sez11['PRO_COM'] = df_sez11['PRO_COM'].astype(int)
    df_sez11['SEZ'] = df_sez11['SEZ'].astype(int)
    
    lat = gdf[gdf['PRO_COM']== id1]['lat'].values[0]
    lon = gdf[gdf['PRO_COM']== id1]['lon'].values[0]

    minx = gdf[gdf['PRO_COM']== id1].geometry.bounds.minx.values[0]
    maxx = gdf[gdf['PRO_COM']== id1].geometry.bounds.maxx.values[0]
    miny = gdf[gdf['PRO_COM']== id1].geometry.bounds.miny.values[0]
    maxy = gdf[gdf['PRO_COM']== id1].geometry.bounds.maxy.values[0]

    region = ee.Geometry.BBox(minx, miny, maxx, maxy)
    
    for mese in mesi:
        nome_img = 'ST_' + comune + '_'+ str(anno) + '0' + str(mese) + '.tif'
        mese2 = ('0' + str(mese+1)) if mese < 9 else str(mese+1)
        
        if not(os.path.isfile(nome_img)):
            
            L9 = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2').filterDate(str(anno) + '-0' + str(mese) +'-01', str(anno) + '-' + mese2 +'-01')
            L8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').filterDate(str(anno) + '-0' + str(mese) +'-01', str(anno) + '-' + mese2 +'-01')
            bands = ['ST_B10', 'QA_PIXEL']
            bandName = ['ST', 'QA_PIXEL']
            filtered_L9 = L9.filter(ee.Filter.lt('CLOUD_COVER', 20)).select(bands, bandName)#.map(cloudMask)
            filtered_L8 = L8.filter(ee.Filter.lt('CLOUD_COVER', 20)).select(bands, bandName)#.map(cloudMask)
            LandsatColl = filtered_L8.merge(filtered_L9)
            LandsatColl = LandsatColl.select('ST').filterBounds(region)

            lst = LandsatColl.max().clip(fc)
            thermal = lst.select('ST.*').multiply(0.00341802).add(149.0).subtract(273.15)


            # zonal stats per sezioni
            print('scarico immagine', comune, anno, mese)


            scarica(thermal, nome_img, region)
            #raster = 'ST_' + comune + .tif"
        else:
            print(nome_img, 'già presente')
        
 # se è il primo mese usa il df delle sezioni originario altrimenti quello con precedenti stats
        #if sdf in locals() or sdf in globals():
        try:
            stats = zonal_stats(sdf, nome_img, stats="mean")
            df_o = sdf
            #print("sdf esistente")
        except:
        #else:   
            stats = zonal_stats(df_o, nome_img, stats="mean")
            df_o = df_sez11
            #print("sdf non esistente")
        
        sdf = pd.DataFrame(stats)
        sdf.rename(columns={'mean': 'm' + str(anno) + '0' + str(mese) }, inplace=True)
        sdf = pd.concat([df_o, sdf], axis=1)
        stats.clear()

        print('regione', reg, comune, 'anno', anno, 'mese', mese)#, center)#, aoi)
    sdf.drop('geometry', inplace=True, axis=1)
    sdf.to_excel(comune + '_' + 'stats.xlsx', index=False) 
    
    
    #del sdf
    
    break


scarico immagine Milano 2023 6


KeyboardInterrupt: 

In [12]:
LandsatColl.size().getInfo()

5

In [56]:
sdf.to_csv(comune + '_' + 'stats.csv', index=False) 

In [24]:
raster = "ST_Milano.tif"
stats = zonal_stats(df_sez11, raster, stats="mean")

In [25]:
df_sez11

,PRO_COM,SEZ2011,SEZ,geometry
0,15146.0,151460005984,5984.0,"MULTIPOLYGON (((9.20533 45.52227, 9.20576 45.5..."
1,15146.0,151460001209,1209.0,"MULTIPOLYGON (((9.22427 45.51616, 9.22448 45.5..."
2,15146.0,151460001062,1062.0,"MULTIPOLYGON (((9.22490 45.49963, 9.22420 45.4..."
3,15146.0,151460005439,5439.0,"MULTIPOLYGON (((9.18993 45.50156, 9.19070 45.5..."
4,15146.0,151460000748,748.0,"MULTIPOLYGON (((9.19610 45.49841, 9.19618 45.4..."
...,...,...,...,...
6080,15146.0,151460002316,2316.0,"MULTIPOLYGON (((9.26795 45.47067, 9.26647 45.4..."
6081,15146.0,151460003011,3011.0,"MULTIPOLYGON (((9.19342 45.42019, 9.19366 45.4..."
6082,15146.0,151460003017,3017.0,"MULTIPOLYGON (((9.19304 45.41848, 9.19434 45.4..."
6083,15146.0,151460003028,3028.0,"MULTIPOLYGON (((9.18418 45.41583, 9.18747 45.4..."


In [40]:
sdf = pd.DataFrame(stats)
sdf = pd.concat([df_sez11, sdf], axis=1)
sdf

,PRO_COM,SEZ2011,SEZ,geometry,mean
0,15146.0,151460005984,5984.0,"MULTIPOLYGON (((9.20533 45.52227, 9.20576 45.5...",46.371600
1,15146.0,151460001209,1209.0,"MULTIPOLYGON (((9.22427 45.51616, 9.22448 45.5...",50.862878
2,15146.0,151460001062,1062.0,"MULTIPOLYGON (((9.22490 45.49963, 9.22420 45.4...",51.254241
3,15146.0,151460005439,5439.0,"MULTIPOLYGON (((9.18993 45.50156, 9.19070 45.5...",48.877008
4,15146.0,151460000748,748.0,"MULTIPOLYGON (((9.19610 45.49841, 9.19618 45.4...",48.545460
...,...,...,...,...,...
6080,15146.0,151460002316,2316.0,"MULTIPOLYGON (((9.26795 45.47067, 9.26647 45.4...",46.867213
6081,15146.0,151460003011,3011.0,"MULTIPOLYGON (((9.19342 45.42019, 9.19366 45.4...",46.003023
6082,15146.0,151460003017,3017.0,"MULTIPOLYGON (((9.19304 45.41848, 9.19434 45.4...",41.976596
6083,15146.0,151460003028,3028.0,"MULTIPOLYGON (((9.18418 45.41583, 9.18747 45.4...",41.103454


In [ ]:
map.choropleth(geo_data = ”King_County_Data/kc_zipcodes.geojson”, data = df_zip_price, columns = [‘ZIPCODE’,’mean_price’], key_on=’feature.properties.ZIPCODE’, fill_color= “YlOrBr”, fill_opacity = 0.6, line_opacity=0.2, legend_name = ‘MEAN SALE PRICE’)
map

In [30]:
vis_params = {
  'min': 30,
  'max': 55,
  'palette': ['purple', 'brown', 'red', 'orange','yellow', 'white']}

# Create a folium map object.
my_map = folium.Map(location=[lat, lon], zoom_start=10, height=500)

my_map.add_ee_layer(-.updateMask(thermal.gt(0)), vis_params, 'Temperature massime registrate')

my_map.add_child(folium.LayerControl())
plugins.Fullscreen().add_to(my_map)
display(my_map)